In [ ]:
import os
cwd_old = os.getcwd()
os.chdir('..')
cwd_new = os.getcwd()
print("The working directory is moved from {} to {}.".format(cwd_old, cwd_new))

# Trim Voice

In [63]:
def save_wav_start_end(ori_wav_path, new_wav_path, start=None, end=None):
    import librosa

    y,fs=librosa.load(ori_wav_path)

    if (start is None) or (math.isnan(start)):
        start = 0
    start_word_index = int(start*fs)
    if (end is None) or (math.isnan(end)):
        end = y.shape[-1] - 1
    end_word_index = int(end*fs)

    _, trim_index = librosa.effects.trim(y, top_db=20)
    if y.shape[-1] != trim_index[-1]:
        print(y.shape)
        print(tuple(trim_index))

    start_trim_index = trim_index[0]
    end_trim_index = trim_index[1]
    #librosa.output.write_wav(new_wav_path, y[start_word_index:],fs)
    # End time are wrong often.

    max_sec_diff = 0.5
    min_sec_diff = 0.1
    start_time_diff = abs(start_word_index - start_trim_index) / fs
    end_time_diff = abs(end_word_index - end_trim_index) / fs
    if (min_sec_diff < start_time_diff) and (start_time_diff < max_sec_diff):
        start_index = max(start_word_index, start_trim_index)
    else:
        start_index = start_trim_index
    if (min_sec_diff < end_time_diff) and (end_time_diff < max_sec_diff):
        end_index = min(end_word_index, end_trim_index)
    else:
        end_index = end_trim_index

    #librosa.output.write_wav(new_wav_path, y[start_index:end_index],fs)
    print("{} sec -> {} sec".format(librosa.get_duration(y), (end_index - start_index) / fs))
    
    return  y[start_index:end_index], fs

In [95]:
import IPython.display as ipd
import librosa

sampling_rate=22050
wav_path = '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/jenie/sleepy/sleepy_1-28_0019.wav'
#wav_path ='/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/jenie/sleepy/sleepy_337-360_0349.wav'
#wav_path = '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/jenie/sleepy/sleepy_169-196_0181.wav'
#wav_path = '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/jenie/sleepy/sleepy_169-196_0194.wav'
#wav_path = '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/jenie/amused/amused_57-84_0078.wav'
#wav_path = '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB-original/EmoV-DB_sorted/sam/sleepy/sleepy_29-56_0030.wav'
wav_path = '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/bea/amused/amused_113-140_0139.wav'
y,fs=librosa.load(wav_path)
y_trim, trim_index = librosa.effects.trim(y, top_db=20)

In [96]:
print(y.shape)
print(y_trim.shape)
print(librosa.get_duration(y), librosa.get_duration(y_trim))

(84480,)
(84480,)
3.8312925170068026 3.8312925170068026


In [97]:
ipd.Audio(y, rate=sampling_rate)

In [98]:
ipd.Audio(y_trim, rate=sampling_rate)

# Check audios

In [75]:
paths = [
 '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/josh/sleepy/sleepy_281-308_0299.wav',
 '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/bea/sleepy/sleepy_309-336_0336.wav',
 '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/bea/amused/amused_85-112_0112.wav',
 '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/sam/amused/amused_281-308_0293.wav',
 '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/bea/sleepy/sleepy_225-252_0240.wav',
 '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/bea/sleepy/sleepy_225-252_0239.wav',
 '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/jenie/angry/angry_421-448_0441.wav',
 '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/sam/disgusted/disgusted_57-84_0060.wav',
 '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/sam/sleepy/sleepy_29-56_0030.wav',
 '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/bea/amused/amused_113-140_0139.wav']

In [74]:
import IPython.display as ipd
import librosa

sampling_rate=22050

wav_path = '/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/sam/sleepy/sleepy_421-448_0421.wav'
y,fs=librosa.load(wav_path)
ipd.Audio(y, rate=sampling_rate)

In [101]:
wav_path = paths[9]
print(wav_path)
y,fs=librosa.load(wav_path)
ipd.Audio(y, rate=sampling_rate)

/data2/sungjaecho/data_tts/EmoV-DB/EmoV-DB/bea/amused/amused_113-140_0139.wav


# Import state_dict

In [33]:
import torch
from model import Tacotron2

In [30]:
model = Tacotron2(hparams).cuda()

In [31]:
pretrained_path = '/data2/sungjaecho/pretrained/tacotron2_statedict.pt'

In [34]:
checkpoint = torch.load(pretrained_path, map_location=lambda storage, loc: storage)

In [35]:
feed_weight = checkpoint['state_dict'].copy()

In [37]:
type(feed_weight)

collections.OrderedDict

In [41]:
for k, v in checkpoint['state_dict'].items():
    print(k)

embedding.weight
encoder.convolutions.0.0.conv.weight
encoder.convolutions.0.0.conv.bias
encoder.convolutions.0.1.weight
encoder.convolutions.0.1.bias
encoder.convolutions.0.1.running_mean
encoder.convolutions.0.1.running_var
encoder.convolutions.0.1.num_batches_tracked
encoder.convolutions.1.0.conv.weight
encoder.convolutions.1.0.conv.bias
encoder.convolutions.1.1.weight
encoder.convolutions.1.1.bias
encoder.convolutions.1.1.running_mean
encoder.convolutions.1.1.running_var
encoder.convolutions.1.1.num_batches_tracked
encoder.convolutions.2.0.conv.weight
encoder.convolutions.2.0.conv.bias
encoder.convolutions.2.1.weight
encoder.convolutions.2.1.bias
encoder.convolutions.2.1.running_mean
encoder.convolutions.2.1.running_var
encoder.convolutions.2.1.num_batches_tracked
encoder.lstm.weight_ih_l0
encoder.lstm.weight_hh_l0
encoder.lstm.bias_ih_l0
encoder.lstm.bias_hh_l0
encoder.lstm.weight_ih_l0_reverse
encoder.lstm.weight_hh_l0_reverse
encoder.lstm.bias_ih_l0_reverse
encoder.lstm.bias_hh_

# Upsamping

In [1]:
import pandas as pd
from hparams import create_hparams

In [2]:
hparams = create_hparams()
split = 'train'

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
columns = ['wav_path', 'text', 'speaker', 'sex', 'emotion', 'lang']
df_list = list()
all_csv_paths = hparams.csv_data_paths.values()
for csv_path in all_csv_paths:
    df = pd.read_csv(csv_path)
    df = df[df.split == split]
    df = df[columns]
    df_list.append(df)
df = pd.concat(df_list, ignore_index=True)

In [4]:
speaker_list = sorted(df.speaker.unique().tolist())
sex_list = sorted(df.sex.unique().tolist())
emotion_list = sorted(df.emotion.unique().tolist())
lang_list = sorted(df.lang.unique().tolist())

In [5]:
print(speaker_list)
print(sex_list)
print(emotion_list)
print(lang_list)

['emovdb-m-josh', 'emovdb-m-sam', 'emovdb-w-bea', 'emovdb-w-jenie', 'ljs-w']
['m', 'w']
['amused', 'angry', 'disgusted', 'neutral', 'sleepy']
['en']


In [6]:
# all_dbs != selected_dbs
all_dbs = hparams.csv_data_paths.keys()
selected_dbs = hparams.dbs
if sorted(list(all_dbs)) != sorted(list(selected_dbs)):
    df_list = list()
    for db in selected_dbs:
        csv_path = hparams.csv_data_paths[db]
        df = pd.read_csv(csv_path)
        df = df[df.split == split]
        df = df[columns]
        df_list.append(df)
    df = pd.concat(df_list, ignore_index=True)

In [7]:
if sorted(emotion_list) != sorted(hparams.emotions):
    df = df[df.emotion.isin(hparams.emotions)]

In [8]:
df_size = df.groupby(['speaker', 'emotion']).size().reset_index(name='size')

In [9]:
df_size.columns

Index(['speaker', 'emotion', 'size'], dtype='object')

In [10]:
df_size

,speaker,emotion,size
0,emovdb-m-josh,amused,280
1,emovdb-m-josh,neutral,284
2,emovdb-m-josh,sleepy,248
3,emovdb-m-sam,amused,471
4,emovdb-m-sam,angry,449
5,emovdb-m-sam,disgusted,477
6,emovdb-m-sam,neutral,474
7,emovdb-m-sam,sleepy,477
8,emovdb-w-bea,amused,277
9,emovdb-w-bea,angry,289


In [11]:
max_size = df_size['size'].max()
print(max_size)

12969


In [12]:
df_dataset_list = list()
for i, row in df_size.iterrows():
    speaker = row['speaker']
    emotion = row['emotion']
    size = row['size']
    
    print(speaker, emotion, size)
    
    n_dup_dfs = max_size // size
    n_rest_samples = max_size % size
    
    df_spk_emo = df[(df.speaker == speaker) & (df.emotion == emotion)]
    
    dup_dfs = [df_spk_emo] * n_dup_dfs + [df[:n_rest_samples]]
    
    df_dataset = pd.concat(dup_dfs, ignore_index=True)
    
    print(df_dataset.shape)
    
    df_dataset_list.append(df_dataset)
    
df_datasets = pd.concat(df_dataset_list, ignore_index=True)
print(df_datasets.shape)

emovdb-m-josh amused 280
(12969, 6)
emovdb-m-josh neutral 284
(12969, 6)
emovdb-m-josh sleepy 248
(12969, 6)
emovdb-m-sam amused 471
(12969, 6)
emovdb-m-sam angry 449
(12969, 6)
emovdb-m-sam disgusted 477
(12969, 6)
emovdb-m-sam neutral 474
(12969, 6)
emovdb-m-sam sleepy 477
(12969, 6)
emovdb-w-bea amused 277
(12969, 6)
emovdb-w-bea angry 289
(12969, 6)
emovdb-w-bea disgusted 316
(12969, 6)
emovdb-w-bea neutral 335
(12969, 6)
emovdb-w-bea sleepy 472
(12969, 6)
emovdb-w-jenie amused 214
(12969, 6)
emovdb-w-jenie angry 464
(12969, 6)
emovdb-w-jenie disgusted 176
(12969, 6)
emovdb-w-jenie neutral 396
(12969, 6)
emovdb-w-jenie sleepy 447
(12969, 6)
ljs-w neutral 12969
(12969, 6)
(246411, 6)


In [19]:
d[('a', 'b')] = 1

In [27]:
str(list(d.keys())[0])

"('speaker', 'emotion')"

In [45]:
import torch

In [47]:
a = torch.zeros(2,4)

In [1]:
a  = list(range(12))

In [22]:
a[0:12:2]

[0, 2, 4, 6, 8, 10]

In [4]:
a

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]